In [63]:
from datetime import date
from pathlib import Path

import numpy as np
import pandas as pd

import requests

In [77]:
URL_cumu = "https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.csv"
URL_daily = "https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.csv"
RAWPATH = Path.cwd().parent / 'raw-data' 
CLEANPATH = Path.cwd().parent / 'clean-data' 

In [65]:
def export_csv(df, name, raw=0):
    if raw:
        path = Path.cwd().parent / 'raw-data' / '{}.csv'.format(name)
    else:
        path = Path.cwd().parent / 'clean-data' / '{}.csv'.format(name)
        
    df.to_csv(path)

In [66]:
def set_nat_cumu(url):
    df_cumu = pd.read_csv(url,sep=";")
    export_csv(df_cumu, "national_cumulative_raw", raw=1)
    
    df_cumu['Date'] = pd.to_datetime(df_cumu['Date_of_report']).dt.date
    df_cumu.drop(['Municipality_code','Municipality_name','Date_of_report'], inplace=True, axis=1)
    
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths'
    }
    
    df_cumu.rename(mapping, inplace=True, axis=1)
    
    df_cumu = df_cumu.groupby(['Date'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df_cumu,"national_cumulative_clean")
    
    print("exported to csv")

In [84]:
def set_province_cumu(path):
    df_province = pd.read_csv(path /'national_cumulative_raw.csv', index_col=0)
    df_province['date'] = pd.to_datetime(df_province['Date_of_report']).dt.date
    df_province.drop(['Municipality_code','Municipality_name','Date_of_report'], inplace=True, axis=1)
    
    mapping = {
        'Total_reported':'total',
        'Hospital_admission':'admitted',
        'Deceased':'deaths',
        'Province':'province'
    }
    
    df_province.rename(mapping, inplace=True, axis=1)
    
    df_province = df_province.groupby(['date','province'])[['total','admitted','deaths']].sum().reset_index()
    
    export_csv(df_province,"province_cumu_clean")
    
    print("exported to csv")

In [74]:
set_nat_cumu(URL_cumu)

exported to csv


In [85]:
set_province_cumu(RAWPATH)

exported to csv


In [80]:
df= pd.read_csv(RAWPATH /'national_cumulative_raw.csv',index_col=0)
df

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased
0,2020-03-13 10:00:00,GM0003,Appingedam,Groningen,0,0,0
1,2020-03-13 10:00:00,GM0010,Delfzijl,Groningen,0,0,0
2,2020-03-13 10:00:00,GM0014,Groningen,Groningen,3,0,0
3,2020-03-13 10:00:00,GM0024,Loppersum,Groningen,0,0,0
4,2020-03-13 10:00:00,GM0034,Almere,Flevoland,1,1,0
...,...,...,...,...,...,...,...
99452,2020-12-08 10:00:00,NaN,NaN,Noord-Holland,295,18,2
99453,2020-12-08 10:00:00,NaN,NaN,Zuid-Holland,1273,35,11
99454,2020-12-08 10:00:00,NaN,NaN,Zeeland,6,2,1
99455,2020-12-08 10:00:00,NaN,NaN,Noord-Brabant,694,23,6


PosixPath('/Users/srdpsrknth/OneDrive/CovidDash/covid19dash/raw-data/national_cumulative_raw.csv')